In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from mysql.connector import connect, Error
from omegaconf import OmegaConf
import sqlalchemy
import pandas as pd
import boto3
from dotenv import load_dotenv

sys.path.append('../src')

In [3]:
load_dotenv()

True

In [4]:
conf = OmegaConf.load('../config.yaml')

In [5]:
mysql_host = os.getenv("MYSQL_HOST")
mysql_user = os.getenv("MYSQL_USER")
mysql_password = os.getenv("MYSQL_PASSWORD")
s3_access_key_id = os.getenv("S3_ACCESS_KEY_ID")
s3_secret_key = os.getenv("S3_SECRET_KEY")

In [6]:
s3 = boto3.client('s3', aws_access_key_id=s3_access_key_id,
                      aws_secret_access_key=s3_secret_key)

In [7]:
from botocore.exceptions import NoCredentialsError

In [8]:
s3_secret_key

'7EfYgoRbbQDL2txRmUya7smsC2tC2LhatfFuxEHULhq2'

In [9]:
s3_access_key_id

'tnU3qtWAh2CgivbuEAEFz4'

In [10]:
try:
    s3.upload_file('../output/bbcrussian_20230118.json', 'gpb_news_external', 'telegram_news/bbcrussian.json')
    print("Upload Successful")
except FileNotFoundError:
    print("The file was not found")
except NoCredentialsError:
    print("Credentials not available")

S3UploadFailedError: Failed to upload ../output/bbcrussian_20230118.json to gpb_news_external/telegram_news/bbcrussian.json: An error occurred (InvalidAccessKeyId) when calling the PutObject operation: The AWS Access Key Id you provided does not exist in our records.

id	channel_id	channel_name	date	text

In [10]:
# CONNECTION TO MYSQL SERVER
try:
    with connect(
        host=conf.host,
        user=conf.user,
        password=conf.password
    ) as connection:
        print(connection)
except Error as e:
    print(e)

ConfigAttributeError: Missing key host
    full_key: host
    object_type=dict

In [9]:
# CREATE DATABASE
db_query = "CREATE DATABASE gpb_news_external"
try:
    with connect(
        host=conf.host,
        user=conf.user,
        password=conf.password,
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(db_query)
except Error as e:
    print(e)

In [13]:
# SHOW DATABASES
query = "SHOW DATABASES"
try:
    with connect(
        host=conf.host,
        user=conf.user,
        password=conf.password,
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(query)
            for db in cursor:
                print(db)
except Error as e:
    print(e)

('MySQL-8300',)
('gpb_news_external',)
('information_schema',)


In [15]:
# CONNECTION TO EXISTING DATABASE
try:
    with connect(
        host=conf.host,
        user=conf.user,
        password=conf.password,
        database=conf.database
    ) as connection:
        print(connection)
except Error as e:
    print(e)

In [18]:
query = \
"""
CREATE TABLE telegram_news(
        id BIGINT,
        channel_id BIGINT,
        channel_name VARCHAR(100),
        date VARCHAR(100),
        text TEXT
)
"""
try:
    with connect(
        host=conf.host,
        user=conf.user,
        password=conf.password,
        database=conf.database
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(query)
            connection.commit()  # after any changes to table
except Error as e:
    print(e)

1050 (42S01): Table 'telegram_news' already exists


In [55]:
show_table_query = "DESCRIBE telegram_news"
try:
    with connect(
        host=conf.host,
        user=conf.user,
        password=conf.password,
        database=conf.database
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(show_table_query)
            result = cursor.fetchall()
            for row in result:
                print(row)
except Error as e:
    print(e)

('id', b'bigint', 'YES', '', None, '')
('channel_id', b'bigint', 'YES', '', None, '')
('channel_name', b'varchar(100)', 'YES', '', None, '')
('date', b'varchar(100)', 'YES', '', None, '')
('text', b'text', 'YES', '', None, '')


In [25]:
df = pd.read_json('../output/bbcrussian.json')
df.head()

,id,channel_id,channel_name,date,text
0,39132,1003921752,bbcrussian,2022-12-19 10:46:30+00:00,Военный корабль Королевских военно-морских сил...
1,39131,1003921752,bbcrussian,2022-12-19 10:25:37+00:00,
2,39130,1003921752,bbcrussian,2022-12-19 10:25:37+00:00,
3,39129,1003921752,bbcrussian,2022-12-19 10:25:36+00:00,
4,39128,1003921752,bbcrussian,2022-12-19 10:25:36+00:00,


In [60]:
try:
    connection = sqlalchemy.create_engine(f'mysql+mysqlconnector://{conf.user}:{conf.password}@{conf.host}/{conf.database}')
    df.to_sql(con=connection, name='telegram_news', if_exists='append', index=False)
except Error as e:
    print(e)

In [41]:
query = "SELECT * FROM telegram_news"
try:
    # with connect(
    #     host=conf.host,
    #     user=conf.user,
    #     password=conf.password,
    #     database=conf.database
    # ) as connection:
    connection = sqlalchemy.create_engine(f'mysql+mysqlconnector://{mysql_user}:{mysql_password}@{mysql_host}/{conf.mysql.database}')
    query_df = pd.read_sql(
        query, 
        connection,
        # chunksize=1000
    )
    # for chunk in query_df:
        # display(chunk)
except Error as e:
    print(e)

In [42]:
query_df.channel_name.value_counts()

meduzalive       54
bbcrussian       50
currenttime      36
varlamov_news    22
bazabazon        15
novaya_pishet     7
nytimes           5
Name: channel_name, dtype: int64

In [43]:
query_df

,rownum,message_id,channel_id,channel_name,channel_url,date,text
0,1,40544,1003921752,bbcrussian,https://t.me/bbcrussian,2023-01-18 22:59:45,Что пишут западные СМИ о новом пакете помощи У...
1,2,40543,1003921752,bbcrussian,https://t.me/bbcrussian,2023-01-18 22:44:52,США выделяют 125 млн долларов на ремонт украин...
2,3,40542,1003921752,bbcrussian,https://t.me/bbcrussian,2023-01-18 22:22:01,Компания Microsoft уволит 10 тыс. своих сотруд...
3,4,40541,1003921752,bbcrussian,https://t.me/bbcrussian,2023-01-18 22:15:12,"Представитель Байдена: США не знают, что могло..."
4,5,40540,1003921752,bbcrussian,https://t.me/bbcrussian,2023-01-18 22:02:01,Суд на Филиппинах оправдал нобелевскую лауреат...
...,...,...,...,...,...,...,...
184,185,15423,1378813139,bazabazon,https://t.me/bazabazon,2023-01-18 11:09:02,Военнослужащий Дмитрий Перов убит при задержан...
185,186,15422,1378813139,bazabazon,https://t.me/bazabazon,2023-01-18 11:02:22,Сбежавший из части с оружием Дмитрий Перов поп...
186,187,15419,1378813139,bazabazon,https://t.me/bazabazon,2023-01-18 10:17:01,МВД Украины сообщает о пяти пострадавших в рез...
187,188,15415,1378813139,bazabazon,https://t.me/bazabazon,2023-01-18 10:09:03,В городе Бровары Киевской области рядом с детс...
